In [5]:
import pandas as pd
import numpy as np
import bokeh

One-hot encoding for categorical columns with get_dummies

In [8]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

Preprocess application_train.csv and application_test.csv

In [33]:
#def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
df = pd.read_csv('/datasets/application_train.csv')
test_df = pd.read_csv('/datasets/application_test.csv')
print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))

Train samples: 307511, test samples: 48744


In [34]:

df = df.append(test_df).reset_index()
# Optional: Remove 4 applications with XNA CODE_GENDER (train set)
df = df[df['CODE_GENDER'] != 'XNA']

# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])
# Categorical features with One-Hot encode
df, cat_cols = one_hot_encoder(df, nan_as_category=True)

# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    #del test_df
    #gc.collect()
    #return df

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_2836\3700531176.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(test_df).reset_index()


Preprocess bureau.csv and bureau_balance.csv

Preprocess previous_applications.csv

Preprocess POS_CASH_balance.csv

Preprocess credit_card_balance.csv

LightGBM GBDT with KFold or Stratified KFold<br>
Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code

In [13]:
df.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,0,0.0,100002,1.0,0,0,0,0,202500.0,406597.5,...,0,0,1,0,0,0.067329,0.498036,202500.0,0.121978,0.060749
1,1,1.0,100003,0.0,1,0,1,0,270000.0,1293502.5,...,0,0,1,0,0,0.070862,0.208736,135000.0,0.132217,0.027598
2,2,2.0,100004,0.0,0,1,0,0,67500.0,135000.0,...,0,1,0,0,1,0.011814,0.500000,67500.0,0.100000,0.050000
3,3,3.0,100006,0.0,1,0,0,0,135000.0,312682.5,...,0,1,0,0,1,0.159905,0.431748,67500.0,0.219900,0.094941
4,4,4.0,100007,0.0,0,0,0,0,121500.0,513000.0,...,0,1,0,0,1,0.152418,0.236842,121500.0,0.179963,0.042623


In [35]:
#def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
train_df = df[df['TARGET'].notnull()]
#test_df = df[df['TARGET'].isnull()]


In [36]:
print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
gc.collect()

Starting LightGBM. Train shape: (307507, 261), test shape: (48744, 121)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\utilisateur\AppData\Local\Temp\ipykernel_2836\2390707251.py", line 2, in <module>
    gc.collect()
NameError: name 'gc' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\utilisateur\Documents\MyAmaWok\OC Data Scientist\Projet OC 7 Implementer un modele de Scoring\.venv\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return Forma

In [21]:
from lightgbm import LGBMClassifier

nthread=4,
n_estimators=10000,
learning_rate=0.02,
num_leaves=34,
colsample_bytree=0.9497036,
subsample=0.8715623,
max_depth=8,
reg_alpha=0.041545473,
reg_lambda=0.0735294,
min_split_gain=0.0222415,
min_child_weight=39.3259775,
silent=-1,
verbose=-1,

In [24]:
from sklearn.model_selection import train_test_split

In [38]:
X_train,X_test,y_train,y_test = train_test_split(train_df.drop('TARGET', axis =1), train_df['TARGET'])

In [39]:
import mlflow
from sklearn.model_selection import GridSearchCV

In [194]:
#erreur avec ponctuation lgb ne pouvait pas focntionner 
X_train_test.columns = X_train_test.columns.str.replace('[#,@,&,:,\,]', '')

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_2836\1913851802.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  X_train_test.columns = X_train_test.columns.str.replace('[#,@,&,:,\,]', '')


Gridsearch + mlflow

In [197]:
#definition gridsearch
grid ={"n_estimators":[100,1000], "max_depth":[10,15]}#{"n_estimators":np.arange(100,500,150), "max_depth":np.arange(2,10,4)}# 
logreg=LGBMClassifier(zero_as_missing =True)
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train_test,y_train)

for i in range(len(logreg_cv.cv_results_['params'])): # pour chaque parametres essayer
    run_index = i
    cv_results = logreg_cv.cv_results_
    with mlflow.start_run(run_name=str(run_index)) as run:  
            mlflow.log_param("folds", logreg_cv.cv)
            print("Logging parameters")
            params = list(logreg_cv.param_grid.keys())
            for param in params:
                mlflow.log_param(param, cv_results["param_%s" % param][run_index])
            print("Logging metrics")
            for score_name in [score for score in cv_results if "mean_test" in score]:
                mlflow.log_metric(score_name, cv_results[score_name][run_index])
                mlflow.log_metric(score_name.replace("mean","std"), cv_results[score_name.replace("mean","std")][run_index])
            #print("Logging model")        
            #mlflow.sklearn.log_model(logreg_cv.best_estimator_, 'logistic_regression',cv_results[score_name.replace("mean","std")][run_index])

Logging parameters
Logging metrics
Logging parameters
Logging metrics
Logging parameters
Logging metrics
Logging parameters
Logging metrics


Display/plot feature importance

In [ ]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [ ]:
def main(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    with timer("Run LightGBM with kfold"):
        feat_importance = kfold_lightgbm(df, num_folds= 10, stratified= False, debug= debug)

In [ ]:
if __name__ == "__main__":
    submission_file_name = "submission_kernel02.csv"
    with timer("Full model run"):
        main()